In [1]:
# Importing all the relevant packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)# To see all the columns of a dataframe
#pd.set_option('display.max_rows', None)

In [2]:
# Function to reduce the memory usage of various Dataframes
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
       
        1. Iterate over every column
        2. Determine if the column is numeric
        3. Determine if the column can be represented by an integer
        4. Find the min and the max value
        5. Determine and apply the smallest datatype that can fit the range of values

    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


In [3]:
def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [4]:
# Loadiing reduced feature training set
X_train = import_data('X_train_final.csv')
y_train = pd.read_csv('y_train.final.csv')

Memory usage of dataframe is 181.24 MB
Memory usage after optimization is: 38.27 MB
Decreased by 78.9%


In [5]:
# Loadiing reduced feature test set
X_test = import_data('X_test_final.csv')
y_test = pd.read_csv('y_test.final.csv')

Memory usage of dataframe is 60.41 MB
Memory usage after optimization is: 12.76 MB
Decreased by 78.9%


In [6]:
# Importing the Sklearn's roc_auc_score module
from sklearn.metrics import roc_auc_score

In [7]:
# Importing  hyperparamater tuning optimizer optuna
import optuna

In [8]:
# Importing required Libraries
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

In [9]:
# Instantiating the Stratified K fold object
cv_strat = StratifiedKFold(5,random_state=42)

## Model_2  : Linear Discriminant Analysis Classifiers

### Computing the test set roc_auc score using Vanilla LDA calssifier with SVD solver with no other hyperparameters to tune.

In [10]:
# Importing LDA Classifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [11]:
# Instantiating the classifier object 
lda_svd = LinearDiscriminantAnalysis(solver='svd')

In [12]:
# Fitting the classifier on the Training data directly as there are no hyperparameters to tune
lda_svd.fit(X_train,y_train)

LinearDiscriminantAnalysis()

In [13]:
# Defining the function to calculate the roc_auc score for the feature sets
def cal_roc_auc(X, y, cls, f_set, t_set, model_name):
    ''' Calculates the roc auc score using the best study parameters 
        f_set : String: specifies 'full feature', 'Reduced feature'
        t_set: String: specifies 'training', 'test'
        model_name: String: specifies Name of the model '''
        
    y_pred = cls.predict_proba(X)
    print('The roc_auc_score for the {} {} set using the best {} is '.format(f_set, t_set, model_name), roc_auc_score(y, y_pred[:,1]))

In [14]:
# Calculating the Reduced feature training set roc_auc score.
cal_roc_auc(X_train, y_train, lda_svd, 'Reduced feature', 'training', 'LDA_SVD Classifier')

The roc_auc_score for the Reduced feature training set using the best LDA_SVD Classifier is  0.752784029146089


In [15]:
# Calculating the Reduced feature test set roc_auc score 
cal_roc_auc(X_test, y_test, lda_svd, 'Reduced feature', 'test', 'LDA_SVD Classifier')

The roc_auc_score for the Reduced feature test set using the best LDA_SVD Classifier is  0.7533073224907925


### Computing the best hyperparameters for the Linear Discriminant Classifier with ‘eigen’ solver using Reduced Feature Training Set.

In [16]:
# Instantiating the classifier object 
lda_eigen = LinearDiscriminantAnalysis(solver='eigen')

In [17]:
# Defining the appropriate objective function for the LDA classifier
def objective_wrappper_lda(X_tr, y_tr, cls=None, cv_strat=None):
    '''
    Optimizes Linear Discriminant Analysis parameters on the given training set X_tr,y_tr
    using cv_strat cross-validation object
    
    '''
    
    def objective(trial):
        params = {
        'shrinkage': trial.suggest_uniform('shrinkage', 0, 1)
        }
        
        cls.set_params(**params)#Initializing the model with the parameters 
    
        return np.mean(cross_val_score(cls, X_tr, y_tr, cv=cv_strat, n_jobs=5, scoring='roc_auc'))  
    return objective

In [18]:
# Defining the evaluation function for study's best parameters
def study_best_score_params(X_tr, y_tr, cls, obj_func, cv_strat, n_trials=100):
    ''' Computes the best hyper parameters of the classsifier and returns 
    Optuna's study's best score & clasifier parameters'''
    study = optuna.create_study(direction='maximize')
    study.optimize(obj_func(X_tr, y_tr, cls, cv_strat), n_trials)
    best_score = study.best_value
    best_params = study.best_params
    return (best_score,best_params)


In [19]:
# Extracting the best model parameters and best study score
best_study_score,best_study_params = study_best_score_params(X_train, y_train, lda_eigen, objective_wrappper_lda, cv_strat, n_trials=200)

[I 2020-12-27 00:00:27,683] A new study created in memory with name: no-name-10dbf9ab-fb61-48b2-b739-12d172a9d95e
[I 2020-12-27 00:00:31,268] Trial 0 finished with value: 0.747848919016227 and parameters: {'shrinkage': 0.25373611555814135}. Best is trial 0 with value: 0.747848919016227.
[I 2020-12-27 00:00:33,625] Trial 1 finished with value: 0.7321350288275011 and parameters: {'shrinkage': 0.7468270076530384}. Best is trial 0 with value: 0.747848919016227.
[I 2020-12-27 00:00:35,906] Trial 2 finished with value: 0.7358668246476705 and parameters: {'shrinkage': 0.6760662976360557}. Best is trial 0 with value: 0.747848919016227.
[I 2020-12-27 00:00:38,163] Trial 3 finished with value: 0.7496216968645756 and parameters: {'shrinkage': 0.14776624477196476}. Best is trial 3 with value: 0.7496216968645756.
[I 2020-12-27 00:00:40,469] Trial 4 finished with value: 0.7085589452499705 and parameters: {'shrinkage': 0.9713598223054516}. Best is trial 3 with value: 0.7496216968645756.
[I 2020-12-27

[I 2020-12-27 00:02:13,651] Trial 43 finished with value: 0.7512311370318948 and parameters: {'shrinkage': 0.030692341568889506}. Best is trial 23 with value: 0.7513543205322551.
[I 2020-12-27 00:02:16,111] Trial 44 finished with value: 0.7505487201135097 and parameters: {'shrinkage': 0.08606679500444125}. Best is trial 23 with value: 0.7513543205322551.
[I 2020-12-27 00:02:18,562] Trial 45 finished with value: 0.7511243287781089 and parameters: {'shrinkage': 0.04143432837944776}. Best is trial 23 with value: 0.7513543205322551.
[I 2020-12-27 00:02:20,968] Trial 46 finished with value: 0.7496455499355122 and parameters: {'shrinkage': 0.1463092329712099}. Best is trial 23 with value: 0.7513543205322551.
[I 2020-12-27 00:02:23,392] Trial 47 finished with value: 0.750163996000839 and parameters: {'shrinkage': 0.11229682378557293}. Best is trial 23 with value: 0.7513543205322551.
[I 2020-12-27 00:02:25,750] Trial 48 finished with value: 0.7478602211280362 and parameters: {'shrinkage': 0.25

[I 2020-12-27 00:04:00,111] Trial 86 finished with value: 0.750284750045547 and parameters: {'shrinkage': 0.10407619321228036}. Best is trial 23 with value: 0.7513543205322551.
[I 2020-12-27 00:04:02,548] Trial 87 finished with value: 0.7513206255246633 and parameters: {'shrinkage': 0.0030037020173083254}. Best is trial 23 with value: 0.7513543205322551.
[I 2020-12-27 00:04:04,921] Trial 88 finished with value: 0.7506789273409764 and parameters: {'shrinkage': 0.0767946716569947}. Best is trial 23 with value: 0.7513543205322551.
[I 2020-12-27 00:04:07,267] Trial 89 finished with value: 0.7512771858543216 and parameters: {'shrinkage': 0.025329004623783662}. Best is trial 23 with value: 0.7513543205322551.
[I 2020-12-27 00:04:09,676] Trial 90 finished with value: 0.7453764208103999 and parameters: {'shrinkage': 0.3790438980060857}. Best is trial 23 with value: 0.7513543205322551.
[I 2020-12-27 00:04:12,033] Trial 91 finished with value: 0.7513529429205488 and parameters: {'shrinkage': 0.0

[I 2020-12-27 00:05:43,815] Trial 129 finished with value: 0.7512580288839047 and parameters: {'shrinkage': 0.00033069472914988834}. Best is trial 23 with value: 0.7513543205322551.
[I 2020-12-27 00:05:46,160] Trial 130 finished with value: 0.7418746985328324 and parameters: {'shrinkage': 0.5156773633869715}. Best is trial 23 with value: 0.7513543205322551.
[I 2020-12-27 00:05:48,656] Trial 131 finished with value: 0.7513359039837628 and parameters: {'shrinkage': 0.015821933376454364}. Best is trial 23 with value: 0.7513543205322551.
[I 2020-12-27 00:05:51,096] Trial 132 finished with value: 0.7512582277845627 and parameters: {'shrinkage': 0.0003430259484581309}. Best is trial 23 with value: 0.7513543205322551.
[I 2020-12-27 00:05:53,532] Trial 133 finished with value: 0.7510273708383385 and parameters: {'shrinkage': 0.049706669260646416}. Best is trial 23 with value: 0.7513543205322551.
[I 2020-12-27 00:05:55,920] Trial 134 finished with value: 0.7511917857461141 and parameters: {'shr

[I 2020-12-27 00:07:27,938] Trial 172 finished with value: 0.7513289816221205 and parameters: {'shrinkage': 0.017454736812988626}. Best is trial 23 with value: 0.7513543205322551.
[I 2020-12-27 00:07:30,423] Trial 173 finished with value: 0.7513230024124805 and parameters: {'shrinkage': 0.018520615831969053}. Best is trial 23 with value: 0.7513543205322551.
[I 2020-12-27 00:07:32,799] Trial 174 finished with value: 0.7511377254761438 and parameters: {'shrinkage': 0.040315953998022076}. Best is trial 23 with value: 0.7513543205322551.
[I 2020-12-27 00:07:35,079] Trial 175 finished with value: 0.751291694928966 and parameters: {'shrinkage': 0.0014825907829626138}. Best is trial 23 with value: 0.7513543205322551.
[I 2020-12-27 00:07:37,558] Trial 176 finished with value: 0.7509212409273105 and parameters: {'shrinkage': 0.05815967869984193}. Best is trial 23 with value: 0.7513543205322551.
[I 2020-12-27 00:07:40,083] Trial 177 finished with value: 0.7512357373123448 and parameters: {'shrin

In [20]:
print('The best roc_auc_score for the study is: ',best_study_score)

The best roc_auc_score for the study is:  0.7513543205322551


In [21]:
print('The best study parameters for the classifier are: ',best_study_params)

The best study parameters for the classifier are:  {'shrinkage': 0.009101434102898598}


In [22]:
# Obtaining the best LDA  model with eigen solver by setting best study parameters.
lda_eigen = lda_eigen.set_params(**best_study_params)

In [23]:
# fitting the best LDA_eigen  model on the reduced feature training set
lda_eigen.fit(X_train, y_train)

LinearDiscriminantAnalysis(shrinkage=0.009101434102898598, solver='eigen')

In [49]:
# Calculating the Reduced feature training set roc_auc score using the best study parameters
cal_roc_auc(X_train, y_train, lda_eigen, 'Reduced feature', 'training', 'LDA Classifier with eigen solver & tuned shrinkage')

The roc_auc_score for the Reduced feature training set using the best LDA Classifier with eigen solver & tuned shrinkage is  0.7527559360951943


In [50]:
# Calculating the Reduced feature test set roc_auc score using the best study parameters
cal_roc_auc(X_test, y_test, lda_eigen, 'Reduced feature', 'test', 'LDA Classifier with eigen solver & tuned shrinkage')

The roc_auc_score for the Reduced feature test set using the best LDA Classifier with eigen solver & tuned shrinkage is  0.7533710812916229


###  Computing the test set roc_auc score using LDA classifier with eigen solver & shrinkage utilizing the Ledoit-Wolf lemma with no other hyperparameters to tune.

In [26]:
# Instantiating the LDA classifier object using  eigen solver utilizing shrinkage using the Ledoit-Wolf lemma
lda_eigen_auto = LinearDiscriminantAnalysis(solver='eigen',shrinkage='auto')

In [27]:
# fitting the best LDA_eigen model with Ledoit-Wolf shrinkage on the reduced feature training set
lda_eigen_auto.fit(X_train, y_train)

LinearDiscriminantAnalysis(shrinkage='auto', solver='eigen')

In [28]:
# Calculating the Reduced feature training set roc_auc score using Ledoit-Wolf shrinkage
cal_roc_auc(X_train , y_train, lda_eigen_auto, 'Reduced feature', 'training', 'LDA Classifier with Ledoit-Wolf shrinkage')

The roc_auc_score for the Reduced feature training set using the best LDA Classifier with Ledoit-Wolf shrinkage is  0.7528107805845184


In [29]:
# Calculating the Reduced feature test set roc_auc score using Ledoit-Wolf shrinkage
cal_roc_auc( X_test, y_test, lda_eigen_auto, 'Reduced feature', 'test', 'LDA Classifier with Ledoit-Wolf shrinkage')

The roc_auc_score for the Reduced feature test set using the best LDA Classifier with Ledoit-Wolf shrinkage is  0.7532314840830818


### Calculating R_R ratio for Vanilla LDA Classifier using SVD solver

In [30]:
# Computing the CV scores using sklearn's cross_val_score
score_lda = cross_val_score(lda_svd, X_train, y_train, cv=cv_strat, n_jobs=5, scoring='roc_auc')

In [31]:
print('The reward associated with the vanilla lda Classifier using roc_auc metric is: ',np.mean(score_lda))

The reward associated with the vanilla lda Classifier using roc_auc metric is:  0.7512082151724193


In [32]:
print('The risk associated with the vanilla lda Classifier using roc_auc metric is: ',np.std(score_lda))

The risk associated with the vanilla lda Classifier using roc_auc metric is:  0.005229724468151239


In [33]:
R_R_Ratio_lda = np.mean(score_lda)/np.std(score_lda)

In [34]:
print('The reward risk ratio for the vanilla lda Classifier using roc_auc metric is: ',R_R_Ratio_lda)

The reward risk ratio for the vanilla lda Classifier using roc_auc metric is:  143.64202545415918


#### The R_R ratio for the vanilla lda Classifier using roc_auc metric is:  143.64202545415918

### Calculating R_R ratio for the best  lda Classifier using eigen solver & tuned shrinkage

In [35]:
# Computing the CV scores using sklearn's cross_val_score
score_lda_eigen = cross_val_score(lda_eigen, X_train, y_train, cv=cv_strat, n_jobs=5, scoring='roc_auc')

In [36]:
print('The reward associated with the best lda classifier with eigen solver using roc_auc metric is: ',np.mean(score_lda_eigen))

The reward associated with the best lda classifier with eigen solver using roc_auc metric is:  0.7513543205322551


In [37]:
print('The risk associated with the best lda classifier with eigen solver using roc_auc metric is: ',np.std(score_lda_eigen))

The risk associated with the best lda classifier with eigen solver using roc_auc metric is:  0.005278518854711489


In [38]:
R_R_Ratio_lda_eigen = np.mean(score_lda_eigen)/np.std(score_lda_eigen)

In [39]:
print('The reward risk ratio for the  best lda classifier with eigen solver using roc_auc metric is: ',R_R_Ratio_lda_eigen)

The reward risk ratio for the  best lda classifier with eigen solver using roc_auc metric is:  142.3418843832703


#### The R_R ratio for the best lda classifier with eigen solver & tuned shrinkage using roc_auc metric is:  142.3418843832703

### Calculating R_R ratio for the lda Classifier with eigen solver & shrinkage utilizing the Ledoit-Wolf lemma

In [40]:
# Computing the CV scores using sklearn's cross_val_score
score_lda_auto = cross_val_score(lda_eigen_auto, X_train, y_train, cv=cv_strat, n_jobs=5, scoring='roc_auc')

In [41]:
print('The reward associated with the lda Classifier utilizing LW lemma shrinkage using roc_auc metric is: ',np.mean(score_lda_auto))

The reward associated with the lda Classifier utilizing LW lemma shrinkage using roc_auc metric is:  0.7513724571407809


In [42]:
print('The risk associated with the lda Classifier utilizing LW lemma shrinkage using roc_auc metric is: ',np.std(score_lda_auto))

The risk associated with the lda Classifier utilizing LW lemma shrinkage using roc_auc metric is:  0.0052036900500323


In [43]:
R_R_Ratio_lda_auto = np.mean(score_lda_auto)/np.std(score_lda_auto)

In [44]:
print('The reward risk ratio for the lda Classifier utilizing LW lemma shrinkage using roc_auc metric is: ',R_R_Ratio_lda_auto)

The reward risk ratio for the lda Classifier utilizing LW lemma shrinkage using roc_auc metric is:  144.39223895284022


In [57]:
# Saving Various LDA models
import joblib
joblib.dump(lda_svd,'Vanilla_lda.joblib')
joblib.dump(lda_eigen,'Lda_eigen.joblib')
joblib.dump(lda_eigen_auto,'Lda_lw_lemma.joblib')

['Lda_lw_lemma.joblib']

#### The R_R ratio for the lda Classifier utilizing LW lemma shrinkage using roc_auc metric is: 144.3922389528402

### R_R Ratio for the best LDA classifier using reduced feature set is:  144.3922389528402, corresponding to  LDA Classifier, utilizing LW lemma Shrinkage.

## Observations:
### 1) All the LDA variants have fitted the dataset pretty well, with no signs of overfitting & test set roc_auc score were well above that of the baseline model.
### 2) The  LDA model with highest test set roc_auc score corresponds to LDA with eigen solver & tuned shrinkage. This model  performed  worse than the tuned Logistic Regression Model on the test set. This was expected as the underlying feature space is not multivariate normal and doesn't have the same covariance matrix for both the classes ,which are the underlying assumption of the LDA models. Further the R_R ratio of the tuned Logistic Regression model was greater that of the LDA classifier using LW lemma Shrinkage.
### 3) As discussed in point 2 , for this dataset, the Logistic Regression model completely dominates all the 3 LDA variants in all respects.

## Model_3 : Quadratic Discriminant Analysis Classifier

### Computing the Reduced Feature test set roc_auc score using QDA classifier 

In [10]:
# Loading reduced feature training set again the usual way as QDA can't handle any data in float16 format
X_train = pd.read_csv('X_train_final.csv')
y_train = pd.read_csv('y_train.final.csv')

In [11]:
# Loading reduced feature test set again the usual way as QDA can't handle any data in float16 format
X_test = pd.read_csv('X_test_final.csv')
y_test = pd.read_csv('y_test.final.csv')

In [12]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [13]:
# Instantiating the QDA classifier object 
QDA = QuadraticDiscriminantAnalysis()

In [14]:
# Defining the appropriate objective function for the QDA classifier
def objective_wrappper_qda(X_tr, y_tr, cls=None, cv_strat=None):
    '''
    Optimizes Quadratic Discriminant Analysis parameters on the given training set X_tr,y_tr
    using cv_strat cross-validation object
    
    '''
    
    def objective(trial):
        params = {
        'reg_param': trial.suggest_uniform('reg_param', 0, 1)
        }
        
        cls.set_params(**params)#Initializing the model with the parameters 
    
        return np.mean(cross_val_score(cls, X_tr, y_tr, cv=cv_strat, n_jobs=5, scoring='roc_auc'))  
    return objective

In [15]:
# Defining the evaluation function for study's best parameters
def study_best_score_params(X_tr, y_tr, cls, obj_func, cv_strat, n_trials=100):
    ''' Computes the best hyper parameters of the classsifier and returns 
    Optuna's study's best score & clasifier parameters'''
    study = optuna.create_study(direction='maximize')
    study.optimize(obj_func(X_tr, y_tr, cls, cv_strat), n_trials)
    best_score = study.best_value
    best_params = study.best_params
    return (best_score,best_params)


In [16]:
# Extracting the best model parameters and best study score
best_study_score,best_study_params = study_best_score_params(X_train, y_train, QDA, objective_wrappper_qda, cv_strat, n_trials=200)

[I 2020-12-27 18:03:12,127] A new study created in memory with name: no-name-430111cf-7b37-46f8-8e8c-77d630328cc1
[I 2020-12-27 18:03:16,891] Trial 0 finished with value: 0.7113357888688814 and parameters: {'reg_param': 0.16578025941058006}. Best is trial 0 with value: 0.7113357888688814.
[I 2020-12-27 18:03:20,601] Trial 1 finished with value: 0.7166289060696854 and parameters: {'reg_param': 0.348181287123548}. Best is trial 1 with value: 0.7166289060696854.
[I 2020-12-27 18:03:24,678] Trial 2 finished with value: 0.7165870507863042 and parameters: {'reg_param': 0.9119226433188083}. Best is trial 1 with value: 0.7166289060696854.
[I 2020-12-27 18:03:28,608] Trial 3 finished with value: 0.7116133585941359 and parameters: {'reg_param': 0.17354396326429955}. Best is trial 1 with value: 0.7166289060696854.
[I 2020-12-27 18:03:32,410] Trial 4 finished with value: 0.7112541631024688 and parameters: {'reg_param': 0.9571423147769414}. Best is trial 1 with value: 0.7166289060696854.
[I 2020-12

[I 2020-12-27 18:06:02,654] Trial 43 finished with value: 0.7225756595813831 and parameters: {'reg_param': 0.7191089736394167}. Best is trial 33 with value: 0.7225773150147644.
[I 2020-12-27 18:06:06,561] Trial 44 finished with value: 0.7179417611922239 and parameters: {'reg_param': 0.4030977699468633}. Best is trial 33 with value: 0.7225773150147644.
[I 2020-12-27 18:06:10,272] Trial 45 finished with value: 0.7218520885719387 and parameters: {'reg_param': 0.6152569673217062}. Best is trial 33 with value: 0.7225773150147644.
[I 2020-12-27 18:06:14,134] Trial 46 finished with value: 0.7208091946362456 and parameters: {'reg_param': 0.546460281251957}. Best is trial 33 with value: 0.7225773150147644.
[I 2020-12-27 18:06:18,120] Trial 47 finished with value: 0.7225642056254035 and parameters: {'reg_param': 0.7266305873897395}. Best is trial 33 with value: 0.7225773150147644.
[I 2020-12-27 18:06:22,195] Trial 48 finished with value: 0.7185766666601887 and parameters: {'reg_param': 0.8846049

[I 2020-12-27 18:08:54,632] Trial 87 finished with value: 0.722095519229032 and parameters: {'reg_param': 0.6352987130770652}. Best is trial 80 with value: 0.7225776508980178.
[I 2020-12-27 18:08:58,646] Trial 88 finished with value: 0.7220478251288421 and parameters: {'reg_param': 0.7862037874403628}. Best is trial 80 with value: 0.7225776508980178.
[I 2020-12-27 18:09:02,480] Trial 89 finished with value: 0.722509529549672 and parameters: {'reg_param': 0.7419988736480618}. Best is trial 80 with value: 0.7225776508980178.
[I 2020-12-27 18:09:06,212] Trial 90 finished with value: 0.7225704745952312 and parameters: {'reg_param': 0.7223274101962175}. Best is trial 80 with value: 0.7225776508980178.
[I 2020-12-27 18:09:10,013] Trial 91 finished with value: 0.7225755160205992 and parameters: {'reg_param': 0.7113759942220782}. Best is trial 80 with value: 0.7225776508980178.
[I 2020-12-27 18:09:13,955] Trial 92 finished with value: 0.7224644382032158 and parameters: {'reg_param': 0.67855783

[I 2020-12-27 18:11:40,993] Trial 130 finished with value: 0.708214901306075 and parameters: {'reg_param': 0.973570428261257}. Best is trial 102 with value: 0.7225777079163622.
[I 2020-12-27 18:11:44,832] Trial 131 finished with value: 0.7225735559107582 and parameters: {'reg_param': 0.7201648742721842}. Best is trial 102 with value: 0.7225777079163622.
[I 2020-12-27 18:11:48,551] Trial 132 finished with value: 0.7225677859767671 and parameters: {'reg_param': 0.7047081171410158}. Best is trial 102 with value: 0.7225777079163622.
[I 2020-12-27 18:11:52,446] Trial 133 finished with value: 0.7223674521211721 and parameters: {'reg_param': 0.7610387161515068}. Best is trial 102 with value: 0.7225777079163622.
[I 2020-12-27 18:11:56,523] Trial 134 finished with value: 0.7223876386113576 and parameters: {'reg_param': 0.6666074203650527}. Best is trial 102 with value: 0.7225777079163622.
[I 2020-12-27 18:12:00,366] Trial 135 finished with value: 0.7225527440709552 and parameters: {'reg_param':

[I 2020-12-27 18:14:26,824] Trial 173 finished with value: 0.7225781299563067 and parameters: {'reg_param': 0.7155771107352454}. Best is trial 173 with value: 0.7225781299563067.
[I 2020-12-27 18:14:30,800] Trial 174 finished with value: 0.7225480092135115 and parameters: {'reg_param': 0.6974503823082697}. Best is trial 173 with value: 0.7225781299563067.
[I 2020-12-27 18:14:34,471] Trial 175 finished with value: 0.7225661067317323 and parameters: {'reg_param': 0.7258895039886075}. Best is trial 173 with value: 0.7225781299563067.
[I 2020-12-27 18:14:38,240] Trial 176 finished with value: 0.7224483585563186 and parameters: {'reg_param': 0.7518690144110949}. Best is trial 173 with value: 0.7225781299563067.
[I 2020-12-27 18:14:42,110] Trial 177 finished with value: 0.7225271404677814 and parameters: {'reg_param': 0.6914802328359675}. Best is trial 173 with value: 0.7225781299563067.
[I 2020-12-27 18:14:46,187] Trial 178 finished with value: 0.7223548489664328 and parameters: {'reg_param

In [17]:
print('The best roc_auc_score for the study is: ',best_study_score)

The best roc_auc_score for the study is:  0.7225781299563067


In [18]:
print('The best study parameters for the classifier are: ',best_study_params)

The best study parameters for the classifier are:  {'reg_param': 0.7155771107352454}


In [19]:
# Obtaining the best reduced feature QDA  model with eigen solver by setting best study parameters.
QDA = QDA.set_params(**best_study_params)

In [20]:
# fitting the best QDA  model on the reduced feature training set
QDA.fit(X_train,y_train)

QuadraticDiscriminantAnalysis(reg_param=0.7155771107352454)

In [24]:
# Calculating the Reduced feature training set roc_auc score using the best study parameters
cal_roc_auc(X_train , y_train, QDA, 'Reduced feature', 'training', 'QDA classifier')

The roc_auc_score for the Reduced feature training set using the best QDA classifier is  0.7258946495984695


In [25]:
# Calculating the Reduced feature test set roc_auc score using the best study parameters
cal_roc_auc(X_test, y_test, QDA, 'Reduced feature', 'test', 'QDA classifier')

The roc_auc_score for the Reduced feature test set using the best QDA classifier is  0.7230740649171299


In [26]:
# Saving the Reduced feature best QDA model 
import joblib
joblib.dump(QDA,'QDA.joblib')

['QDA.joblib']

### Calculating R_R ratio for best QDA Classifier.

In [28]:
# Computing the CV scores using sklearn's cross_val_score
score_qda = cross_val_score(QDA, X_train, y_train, cv=cv_strat, n_jobs=5, scoring='roc_auc')

In [29]:
print('The reward associated with the best QDA Classifier using roc_auc metric is: ',np.mean(score_qda))

The reward associated with the best QDA Classifier using roc_auc metric is:  0.7225781299563067


In [30]:
print('The risk associated with the best QDA Classifier using roc_auc metric is: ',np.std(score_qda))

The risk associated with the best QDA Classifier using roc_auc metric is:  0.006074359365354742


In [31]:
R_R_Ratio_qda = np.mean(score_qda)/np.std(score_qda)

In [32]:
print('The reward risk ratio for the best QDA Classifier using roc_auc metric is: ',R_R_Ratio_qda)

The reward risk ratio for the best QDA Classifier using roc_auc metric is:  118.95544640930348


#### R_R Ratio for the best QDA classifier using roc_auc metric is:  118.95544640930348

## Observations:
### 1) The tuned QDA model has the worst test set roc_auc score as well as R_R ratio of all the fitted models till now. This was expected as the underlying Feature space is not multivariate normal, which is the underlying assumption of the QDA model. Any departure from normality affects the QDA more than the LDA, which is also observed here.
### 2) Owing to both very low  R_R ratio and test set roc_auc, we can rule out the QDA model for this dataset.

### R_R Ratio for the best classifier  in Discriminant Analysis family utilizing  roc_auc metric is:   144.3922389528402, corresponding to LDA Classifier, with LW lemma Shrinkage.

## Thus keeping everything into account, the best discriminant classifier is Vanilla LDA classifier  with SVD solver for this dataset.